# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Mathew  ->  J. Mathew  |  ['J. Mathew']
J. Liu  ->  J. Liu  |  ['J. Liu']
F. Walter  ->  F. Walter  |  ['F. Walter']
T. Henning  ->  T. Henning  |  ['T. Henning']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
Arxiv has 71 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2412.03690
extracting tarball to tmp_2412.03690...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.03816
extracting tarball to tmp_2412.03816...

 done.
Retrieving document from  https://arxiv.org/e-print/2412.04088


extracting tarball to tmp_2412.04088... done.
Retrieving document from  https://arxiv.org/e-print/2412.04171


...

Found 67 bibliographic references in tmp_2412.04171/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2412.04224


extracting tarball to tmp_2412.04224...

 done.


S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']


Found 103 bibliographic references in tmp_2412.04224/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.04171-b31b1b.svg)](https://arxiv.org/abs/2412.04171) | **Deep imaging of three accelerating stars using SHARK-NIR and LMIRCam at LBT**  |
|| D. Mesa, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-12-06*|
|*Comments*| *12 pages, 9 figures, accepted for publication on MNRAS*|
|**Abstract**|            The combination of detection techniques enhances our ability to identify companions orbiting nearby stars. We employed high-contrast imaging to constrain mass and separation of possible companions responsible for the significant proper motion anomalies of the nearby stars HIP 11696, HIP 47110 and HIP 36277. These targets were observed using the LBT's high-contrast camera, SHARK-NIR, in H-band using a Gaussian coronagraph, and with the LMIRCam instrument in the L'-band and using a vAPP coronagraph. Both observations were conducted simultaneously. Additionally, constraints at short separations from the host star are derived analyzing the renormalized unit weight error (RUWE) values from the Gaia catalogue. We find that the companion responsible for the anomaly signal of HIP 11696 is likely positioned at a distance from 2.5 to 28 astronomical units from its host. Its mass is estimated to be between 4 and 16 Jupiter masses, with the greater mass possible only at the upper end of the separation range. Similar limits were obtained for HIP 47110 where the companion should reside between 3 and 30 au with a mass between 3 and 10 MJup. For HIP 36277, we identified a faint stellar companion at large separation, though it might be substellar depending on the assumed age for the star. Considering the older age, this object accounts for the absolute value of the PMa vector but not for its direction. Additionally, we found a substellar candidate companion at a closer separation that could explain the PMa signal, considering a younger age for the system.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.04224-b31b1b.svg)](https://arxiv.org/abs/2412.04224) | **The radio properties of the JWST-discovered AGN**  |
|| G. Mazzolari, et al. -- incl., <mark>S. Belladitta</mark> |
|*Appeared on*| *2024-12-06*|
|*Comments*| *Submitted to A&A, comments are welcome*|
|**Abstract**|            We explore the radio emission of spectroscopically confirmed, X-ray weak, Broad Line AGN (BLAGN, or type 1) selected with JWST in the GOODS-N field, one of the fields with the best combination of deep radio observations and statistics of JWST-selected BLAGN. We use deep radio data at different frequencies (144\,MHz, 1.5\,GHz, 3\,GHz, 5.5\,GHz, 10\,GHz), and we find that none of the 22 sources investigated is detected at any of the aforementioned frequencies. Similarly, the radio stacking analysis does not reveal any detection down to an rms of $\sim 0.2\mu$Jy beam$^{-1}$, corresponding to a $3\sigma$ upper limit at rest frame 5 GHz of $L_{5GHz}=2\times10^{39}$ erg s$^{-1}$ at the mean redshift of the sample $z\sim 5.2$. We compared this and individual sources upper limits with expected radio luminosities estimated assuming different AGN scaling relations. For most of the sources the radio luminosity upper limits are still compatible with expectations for radio-quiet (RQ) AGN; nevertheless, the more stringent stacking upper limits and the fact that no detection is found would suggest that JWST-selected BLAGN are weaker than standard AGN even at radio frequencies. We discuss some scenarios that could explain the possible radio weakness, such as free-free absorption from a dense medium, or the lack of either magnetic field or a corona, possibly as a consequence of super-Eddington accretion. These scenarios would also explain the observed X-ray weakness. We also conclude that $\sim$1 dex more sensitive radio observations are needed to better constrain the level of radio emission (or lack thereof) for the bulk of these sources. The Square Kilometer Array Observatory (SKAO) will likely play a crucial role in assessing the properties of this AGN population.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.03690-b31b1b.svg)](https://arxiv.org/abs/2412.03690) | **Constraining the link between the 2175{\AA} dust absorption feature and PAHs in Nearby Star-Forming Galaxies using Swift/UVOT and JWST/MIRI**  |
|| A. J. Battisti, et al. -- incl., <mark>J. Mathew</mark> |
|*Appeared on*| *2024-12-06*|
|*Comments*| *26 pages, 19 figures, 4 tables. Accepted for publication in PASA*|
|**Abstract**|            The 2175Å bump is a prominent absorption feature at ultraviolet (UV) wavelengths in dust extinction and attenuation curves. Understanding the relative strength of this feature is important for accurate dust corrections at both low- and high-redshift. This feature is postulated to arise from polycyclic aromatic hydrocarbon (PAH) dust grains; however, the carrier has not been definitively established. We present results on the correlation between the 2175Å feature and PAH abundances in a spatially-resolved manner for 15 local galaxies in the PHANGS-JWST survey that have NUV and mid-IR imaging data from Swift/UVOT and JWST/MIRI, respectively. We find a moderate positive correlation between the 2175Å feature strength and PAH abundance, albeit with large intrinsic scatter. However, most of this trend can be attributed to a stronger negative correlation of both quantities with SFR surface density and specific-SFR (proxies of ionising radiation). The latter trends are consistent with previous findings that both the 2175Å carrier and PAHs are small grains that are easily destroyed by UV photons, although the proxy for PAH abundance could also be influenced by dust heating. When controlling for SFR surface density, we find weaker correlations between the 2175Å feature and PAH abundances, disfavouring a direct link. However, analyses based on spectroscopic measurements of the 2175Å feature and PAH features are required to verify our findings. No significant trends with gas-phase metallicity are found for the 2175Å feature and PAHs, however the metallicity range of our sample is limited. We provide prescriptions for the strength of the 2175Å feature and PAHs in local massive (metal-rich) galaxies with SFR surface density and specific-SFR, however the former should be used with caution since bump strengths measured from Swift/UVOT are expected to be underestimated.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.03816-b31b1b.svg)](https://arxiv.org/abs/2412.03816) | **Photospheric Swirls in a Quiet-Sun Region**  |
|| Q. Xie, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-12-06*|
|*Comments*| *Accepted for publication in the Astrophysical Journal. 13 pages, 10 figures, 1 table*|
|**Abstract**|            Swirl-shaped flow structures have been observed throughout the solar atmosphere, in both emission and absorption, at different altitudes and locations, and are believed to be associated with magnetic structures. However, the distribution patterns of such swirls, especially their spatial positions, remain unclear. Using the Automated Swirl Detection Algorithm (ASDA), we identified swirls from the high-resolution photospheric observations, centered on Fe I 630.25 nm, of a quiet region near the Sun's central meridian by the Swedish 1-m Solar Telescope. Through a detailed study of the locations of the detected small-scale swirls with an average radius of $\sim$300 km, we found that most of them are located in lanes between mesogranules (which have an average diameter of $\sim$5.4 Mm) instead of the commonly believed intergranular lanes. The squared rotation, expansion/contraction, vector speeds, and proxy kinetic energy are all found to follow Gaussian distributions. Their rotation speed, expansion/contraction speed, and circulation are positively correlated with their radius. These results suggest that photospheric swirls at different scales and locations across the observational 56.5" $\times$ 57.5" field-of-view (FOV) could share the same triggering mechanism at preferred spatial and energy scales. A comparison with previous work suggests that the number of photospheric swirls is positively correlated with the number of local magnetic concentrations, stressing the close relation between swirls and local magnetic concentrations:the number of swirls should positively correlate with the number and strength of local magnetic concentrations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2412.04088-b31b1b.svg)](https://arxiv.org/abs/2412.04088) | **A Spectrophotometric analysis and dust properties of classical nova V5584 Sgr**  |
|| M. S. Bisht, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-12-06*|
|*Comments*| *13 pages, 10 figures, 5 tables, Accepted for publication in MNRAS. arXiv admin note: text overlap with arXiv:2408.01924*|
|**Abstract**|            In this work, optical observations of the nova V5584 Sgr are presented. These observations cover different phases including pre-maximum, early decline, and nebular. The spectra are dominated by hydrogen Balmer, Fe II, and O I lines with P-Cygni profiles in the early phase, which are subsequently observed in complete emission. The presence of numerous Fe II lines and low ejecta velocity aligns with the Fe II type nova classification. From optical and NIR colors it is clear that this nova manifests dust formation in the ejecta. The dust temperature and mass were estimated from a spectral energy distribution (SED) fit to the JHK band magnitudes and the WISE data. Light curve analysis shows t$_2$ and t$_3$ values of $\sim$ 26 and $\sim$ 48 days, classifying the nova as moderately fast. The physical and chemical properties during early decline and later phases were evaluated using the photoionization code CLOUDY. The best-fit model parameters from two epochs of multiwavelength spectra are compatible with a hot white dwarf source with a roughly constant luminosity of $\sim$ (2.08 $\pm$ 0.10) $\times$ 10$^{36}$ erg s$^{-1}$. We find an ejected mass of $\sim$ (1.59 $\pm$ 0.04) $\times$ 10$^{-4}$M$_{\odot}$. Abundance analysis indicates that the ejecta is significantly enriched relative to solar values, with O/H = 30.2, C/H = 10.8, He/H = 1.8, Mg/H = 1.68, Na/H = 1.55, and N/H = 45.5 in the early decline phase, and O/H = 4.5, Ne/H = 1.5, and N/H = 24.5 in the nebular phase.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2412.04171/./HIP11696_comparemodels_HL.png', 'tmp_2412.04171/./HIP47110_comparemodels_HL.png', 'tmp_2412.04171/./HIP36277_comparemodels_HL_41Myr.png', 'tmp_2412.04171/./HIP36277_comparemodels_HL_5Gyr.png', 'tmp_2412.04171/./HIP47110_PMaplot_112Myr.png', 'tmp_2412.04171/./plot_astro_HIP11696_test2.png']
copying  tmp_2412.04171/./HIP11696_comparemodels_HL.png to _build/html/
copying  tmp_2412.04171/./HIP47110_comparemodels_HL.png to _build/html/
copying  tmp_2412.04171/./HIP36277_comparemodels_HL_41Myr.png to _build/html/
copying  tmp_2412.04171/./HIP36277_comparemodels_HL_5Gyr.png to _build/html/
copying  tmp_2412.04171/./HIP47110_PMaplot_112Myr.png to _build/html/
copying  tmp_2412.04171/./plot_astro_HIP11696_test2.png to _build/html/
exported in  _build/html/2412.04171.md
    + _build/html/tmp_2412.04171/./HIP11696_comparemodels_HL.png
    + _build/html/tmp_2412.04171/./HIP47110_comparemodels_HL.png
    + _build/html/tmp_2412.04171/./HIP36277_comparemodels_HL_41Myr

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\MJup}{M_{\mathrm{Jup}}\xspace}$
$\newcommand{\RJup}{R_{\mathrm{Jup}}}$
$\newcommand{\RSun}{R_{\odot} }$
$\newcommand{\MSun}{M_{\odot} }$
$\newcommand{\LSun}{L_{\odot} }$
$\newcommand{\teff}{T_{e\!f\!f} }$
$\newcommand{\MEarth}{M_{\oplus} }$
$\newcommand{\REarth}{R_{\oplus} }$
$\newcommand{\logg}{log~\emph{g} }$
$\newcommand{\mic}{\mum }$
$\newcommand{\as}{\hbox{^{\prime\prime}} }$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Deep imaging of three accelerating stars using SHARK-NIR and LMIRCam at LBT

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.04171-b31b1b.svg)](https://arxiv.org/abs/2412.04171)<mark>Appeared on: 2024-12-06</mark> -  _12 pages, 9 figures, accepted for publication on MNRAS_

</div>
<div id="authors">

D. Mesa, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** The combination of detection techniques enhances our ability to identify companions orbiting nearby stars. We employed high-contrast imaging to constrain mass and separation of possible companions responsible for the significant proper motion anomalies of the nearby stars HIP 11696, HIP 47110 and HIP 36277. These targets were observed using the LBT’s high-contrast camera, SHARK-NIR, in H-band using a Gaussian coronagraph, and with the LMIRCam instrument in the L’-band and using a vAPP coronagraph. Both observations were conducted simultaneously. Additionally, constraints at short separations from the host star are derived analyzing the renormalized unit weight error (RUWE) values from the Gaia catalogue. We find that the companion responsible for the anomaly signal of HIP 11696 is likely positioned at a distance from 2.5 to 28 astronomical units from its host. Its mass is estimated to be between 4 and 16 Jupiter masses, with the greater mass possible only at the upper end of the separation range. Similar limits were obtained for HIP 47110 where the companion should residebetween 3 and  30 au with a mass between 3 and 10 MJup. For HIP 36277, we identified a faint stellar companion at large separation, though it might be substellar depending on the assumed age for the star. Considering the older age, this object accounts for the absolute value of the PMa vector but not for its direction. Additionally, we found a substellar candidate companion at a closer separation that could explain the PMa signal, considering a younger age for the system.

</div>

<div id="div_fig1">

<img src="tmp_2412.04171/./HIP11696_comparemodels_HL.png" alt="Fig9.1" width="25%"/><img src="tmp_2412.04171/./HIP47110_comparemodels_HL.png" alt="Fig9.2" width="25%"/><img src="tmp_2412.04171/./HIP36277_comparemodels_HL_41Myr.png" alt="Fig9.3" width="25%"/><img src="tmp_2412.04171/./HIP36277_comparemodels_HL_5Gyr.png" alt="Fig9.4" width="25%"/>

**Figure 9. -** Comparison of mass limits for HIP 11696 (upper left panel), HIP 47110 (upper right panel) and HIP 36277 (bottom panels) assuming different atmospheric models both for the H (solid lines) and the L (dashed lines) spectral band. For the case of HIP 36277 we considered both the younger age of 41 Myr (bottom left panel) and the older age of $\sim$5 Gyr (bottom right panel). In the latter case, as detailed in the text we could use just a lower number of models. For HIP 36277 we also included the calculated position for the inner faint candidate companion represented by a red square. (*f:modelhip11696*)

</div>
<div id="div_fig2">

<img src="tmp_2412.04171/./HIP47110_PMaplot_112Myr.png" alt="Fig3" width="100%"/>

**Figure 3. -** Comparison of the plot of mass limits as a function of the separation from the host star to explain the PMa measurement at the Gaia eDR3 epoch (blue lines) for HIP 47110 with the limits in mass obtained from the SHARK-NIR observations and the AMES-COND atmospheric models (orange solid line). The green and the red solid lines represent the mass limits obtained using the minimum and the maximum ages for the star, respectively. Finally, the brown solid line displays the mass limit at a short separation from the star that can be deduced from considerations of the RUWE value. (*f:pmahip47110*)

</div>
<div id="div_fig3">

<img src="tmp_2412.04171/./plot_astro_HIP11696_test2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Relative astrometric position for the candidate companion detected around HIP 11696. The red circle represents the relative position of the object at the NIRC2/KeckII observation epoch, while the green square represents the relative position of the candidate companion at the NIRI observation epoch. Finally, the violet diamond represents the relative position at the epoch of the LBT observation. The error bars on the positions at all epochs are not visible in this plot because of their small dimensions. The solid black line represents the expected relative movement for a background object. The black square is the expected position for a background object at the epoch of the SHARK-NIR and LMIRCam observation. (*f:astrohip11696*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.04171"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HII}{H {\sc ii}\rm }$
$\newcommand{\NII}{{[N {\sc ii}] }}$
$\newcommand{\NIIs}{{[N {\sc ii}] }}$
$\newcommand{\NVs}{{[N {\sc v}] }}$
$\newcommand{\NIVl}{{N {\sc iv}] \lambda}}$
$\newcommand{\NIVll}{{N {\sc iv}] \lambda\lambda}}$
$\newcommand{\NVl}{{N {\sc v} \lambda}}$
$\newcommand{\NIIl}{{[N {\sc ii}] \lambda}}$
$\newcommand{\NIIll}{{[N {\sc ii}] \lambda\lambda}}$
$\newcommand{\NeIII}{{[Ne {\sc iii} ]}}$
$\newcommand{\NeIIIs}{{[Ne {\sc iii}] }}$
$\newcommand{\NeIIIl}{{[Ne {\sc iii}] \lambda}}$
$\newcommand{\NeIV}{{[Ne {\sc iv}] }}$
$\newcommand{\NeIVs}{{[Ne {\sc iv}] }}$
$\newcommand{\NeIVl}{{[Ne {\sc iv}] \lambda}}$
$\newcommand{\NeIVll}{{[Ne {\sc iv}] \lambda\lambda}}$
$\newcommand{\NeV}{{[Ne {\sc v}] }}$
$\newcommand{\NeVs}{{[Ne {\sc v}] }}$
$\newcommand{\NeVl}{{[Ne {\sc v}] \lambda}}$
$\newcommand{\SII}{{[S {\sc ii}] }}$
$\newcommand{\SIIs}{{[S {\sc ii}] }}$
$\newcommand{\SIIl}{{[S {\sc ii}] \lambda}}$
$\newcommand{\SIIll}{{[S {\sc ii}] \lambda\lambda}}$
$\newcommand{\SIII}{{[S {\sc iii}] }}$
$\newcommand{\SIIIs}{{[S {\sc iii}] }}$
$\newcommand{\SIIIl}{{[S {\sc iii}] \lambda}}$
$\newcommand{\OIII}{{[O {\sc iii}] }}$
$\newcommand{\OIIIs}{{[O {\sc iii}] }}$
$\newcommand{\OIIIsf}{{O {\sc iii}] }}$
$\newcommand{\OIIIl}{{[O {\sc iii}] \lambda}}$
$\newcommand{\OIIIll}{{[O {\sc iii}] \lambda\lambda}}$
$\newcommand{\OII}{{[O {\sc ii}] }}$
$\newcommand{\OIIs}{{[O {\sc ii}] }}$
$\newcommand{\OIIl}{{[O {\sc ii}] \lambda}}$
$\newcommand{\OIIll}{{[O {\sc ii}] \lambda\lambda}}$
$\newcommand{\OI}{{[O {\sc i}] }}$
$\newcommand{\OIs}{{[O {\sc i}] }}$
$\newcommand{\OIl}{{[O {\sc i}] \lambda}}$
$\newcommand{\CII}{{[C {\sc ii}] }}$
$\newcommand{\CIIs}{{C {\sc ii}] }}$
$\newcommand{\CIII}{{C {\sc iii}] }}$
$\newcommand{\CIIIs}{{C {\sc iii}] }}$
$\newcommand{\CIIIll}{\CIIIs\lambda\lambda}$
$\newcommand{\CIV}{{C {\sc iv} }}$
$\newcommand{\CIVs}{{C {\sc iv} }}$
$\newcommand{\CIVll}{{C {\sc iv} \lambda\lambda}\xspace}$
$\newcommand{\CIVl}{{[C {\sc iv} \lambda}\xspace}$
$\newcommand{\HeII}{{He {\sc ii} }}$
$\newcommand{\HeIIl}{{He {\sc ii} \lambda}}$
$\newcommand{\FeII}{{[Fe {\sc ii}] }}$
$\newcommand{\Ha}{H\alpha }$
$\newcommand{\Has}{H\alpha }$
$\newcommand{\Hb}{H\beta }$
$\newcommand{\Hbs}{H\beta }$
$\newcommand{\Hg}{H\gamma }$
$\newcommand{\Lya}{Ly\alpha }$
$\newcommand{\gm}[1]{{\color{black}{\bf[GM: #1]}}}$
$\newcommand{\RMcomm}[1]{{\color{teal}[RM: #1]}}$</div>



<div id="title">

# The radio properties of the JWST-discovered AGN

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2412.04224-b31b1b.svg)](https://arxiv.org/abs/2412.04224)<mark>Appeared on: 2024-12-06</mark> -  _Submitted to A&A, comments are welcome_

</div>
<div id="authors">

G. Mazzolari, et al. -- incl., <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** The _James Webb Space Telescope_ (JWST) has discovered a large population of Active Galactic Nuclei (AGN) in the early Universe. A large fraction of these AGN revealed a significant lack of X-ray emission, with observed X-ray luminosity upper limits 2-3 dex lower than expected.  We explore the radio emission of these AGN, focusing on the JWST-selected Broad Line AGN (BLAGN, or type 1) in the GOODS-N field, one of the fields with the best combination of deep radio observations and statistics of JWST-selected, spectroscopically confirmed BLAGN. We use deep radio data at different frequencies (144 MHz, 1.5 GHz, 3 GHz, 5.5 GHz, 10 GHz), and we find that none of the 22 sources investigated is detected at any of the aforementioned frequencies.  Similarly, the radio stacking analysis does not reveal any detection down to an rms of $\sim 0.2\mu$ Jy beam $^{-1}$ , corresponding to a $3\sigma$ upper limit at rest frame 5 GHz of $L_{5GHz}=2\times10^{39}$ erg s $^{-1}$ at the mean redshift of the sample $z\sim 5.2$ . We compared this and individual sources upper limits with expected radio luminosities estimated assuming different AGN scaling relations. For most of the sources the radio luminosity upper limits are still compatible with expectations for radio-quiet (RQ) AGN; nevertheless, the more stringent stacking upper limits and the fact that no detection is found would suggest that JWST-selected BLAGN are weaker than standard AGN even at radio frequencies.  We discuss some scenarios that could explain the possible radio weakness, such as free-free absorption from a dense medium, or the lack of either magnetic field or a corona, possibly as a consequence of super-Eddington accretion. These scenarios would also explain the observed X-ray weakness. We also conclude that $\sim$ 1 dex more sensitive radio observations are needed to better constrain the level of radio emission (or lack thereof) for the bulk of these sources. The Square Kilometer Array Observatory (SKAO) will likely play a crucial role in assessing the properties of this AGN population.

</div>

<div id="div_fig1">

<img src="tmp_2412.04224/./figure/4panel_flip_3GHz_alpha-0.5_5GHz.jpeg" alt="Fig7" width="100%"/>

**Figure 7. -** Observed rest-frame 5GHz radio luminosity versus expected rest-frame 5GHz radio luminosity of different samples of high-z and local BLAGN according to the four RQ AGN relations described in Sect. \ref{sec:Lradexp}. The black data points represent the JWST-detected BLAGN analyzed in this work, the red diamond their stack, the gold star the BLAGN GN-28074 reported in [Juod\vzbalis, Ji and Maiolino (2024)](). The green symbols indicate sources from different local analog samples. The green square represents the position of SBS 0355-052  ([Hatano, Ouchi and Nakajima 2023](), [Johnson, Hunt and Reines 2009]()) , the thick crosses represent the metal-poor dwarf BLAGN reported in [Burke, et. al (2021)](), the thin crosses the X-ray weak BLAGN reported in [Paul, Plotkin and Brandt (2024)](), the filled triangles the radio detected and RQ NLS1 reported in [Berton, Congiu and Järvelä (2018)](). The light blue data point in the top-left panel represents the stack of the X-ray selected but radio undetected AGN on the GOODS-N field performed by [Radcliffe, Barthel and Garrett (2021)](). The red diamond and the black data points in the bottom-right panel, are computed considering $\log R=1$, while the red circle indicate the expected radio luminosity assuming $\log R=0$. The gray shaded areas represent the scatter of the relations, while the black dashed line is the 1:1 relation. (*fig:RQ_panel*)

</div>
<div id="div_fig2">

<img src="tmp_2412.04224/./figure/SFRdata-SFRMS.jpeg" alt="Fig2" width="100%"/>

**Figure 2. -** Ratio between the SFR derived from the radio luminosity upper limits and the SFR derived assuming the MS of [Popesso, Concas and Cresci (2023)](). The red data point corresponds to the value of the stack. The black dashed line traces the threshold below which the $SFR_{MS}$ becomes incompatible with the radio undetection. (*fig:SFR_MS*)

</div>
<div id="div_fig3">

<img src="tmp_2412.04224/./figure/SFRdata-SFRHa.jpeg" alt="Fig4" width="100%"/>

**Figure 4. -** Ratio between the SFR derived from the radio luminosity upper limits and the SFR derived assuming the whole $\Ha$ emission (narrow+broad) to be due to SF. The red data point corresponds to the value of the stack. The black dashed line traces the threshold below which the $SFR_{H\alpha}$ becomes incompatible with the radio undetection. (*fig:SFR_Ha*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2412.04224"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

270  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
